Q: В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации? Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы.

A: Поскольку коллаборативная фильтрация основана на матричной факторизации, она учитывает только реальные user-item взаимодействия (купил/не купил), игнорируя остальные доступные параметры (бренды товаров, время совершения покупки, частота покупок, возраст пользователей, их социальный статус, etc). В связи с этим возникает проблема "холодного старта": выдачи рекомендаций новым или мало активным пользователям, а также рекомендация новых товаров. И в предыдущих работах этого курса можно заметить, что некоторые алгоритмы выдают меньше предсказаний, чем необходимо. Гибридная рекомендательная система решает эти проблемы.

Q: Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/ Нам интересна именно рекомендательная система, раздел “Производительность системы” можно пропустить Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

A: Описанная рекомендательная система HH состоит (в общем виде) из 4-х элементов (см. рис. Схема работы рекомендательной системы). На вебинарах мы рассматривали состоящие только из 2-х (ALS similar-user + item-item). К сожалению, в статье нет подробного описания работы каждого этапа моделирования, но в качестве примера сложностей в продакшене предположу следующее: эвристический фильтр на 2-х признаках и фльтрующая модель 1 - на 4-х - работают, скорее всего быстро, но насколько качественно - вопрос. Также не очень понятно, как решается проблема индексации и генерации признаков для неполных или некорректно заполненных вакансий/резюме.

Q: На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist’а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:
1) Какой датасет используют авторы?
2) Что используют в качестве признаков?
3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко

# practice

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import bm25_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

from additional import DataProcessor
from functools import partial
from sklearn.preprocessing import StandardScaler

## load & split

In [2]:
# load purchases
purchases = pd.read_csv('retail_train.csv')

# train/test split
test_size_weeks = 3
train = purchases[purchases['week_no'] < purchases['week_no'].max() - test_size_weeks].copy()
test = purchases[purchases['week_no'] >= purchases['week_no'].max() - test_size_weeks].copy()

# prepare true values
true_values = test.groupby('user_id')['item_id'].unique().reset_index()
true_values.columns=['user_id', 'actual']

In [3]:
# load & prepare products info
item_features = pd.read_csv('product.csv')
item_features.columns = item_features.columns.str.lower()
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

# load & prepare users info
user_features = pd.read_csv('hh_demographic.csv')
user_features.columns = user_features.columns.str.lower()
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

## product features EDA

## user features EDA

## prepare dataset

подготовим параметры обработки датасета:
* defaults: на основе кол-ва проданных товаров
* mix_feat: на комбинации стоимости и кол-ва проданных товаров

In [4]:
mix_feat_params = {
    'top_config': {'fields': ['quantity', 'sales_value'],
                   'beta': [1., 1.],
                   'k': 5000,
                   'scaler': StandardScaler},
    'uim_config': {'aggfunc': 'sum',
                   'weights': None},
}

defaults_params = {
    'top_config': {'fields': ['quantity'],
                   'k': 5000},
    'uim_config': {'aggfunc': 'sum',
                   'weights': None},
}

In [5]:
# weighted_mix_feat_params = {
#     'top_config': {'fields': ['quantity', 'sales_value'],
#                    'beta': [1., 1.],
#                    'k': 5000,
#                    'scaler': StandardScaler},
#     'uim_config': {'aggfunc': 'sum',
#                    'weights': bm25_weight},
# }
#
# weighted_defaults_params = {
#     'top_config': {'fields': ['quantity'],
#                    'k': 5000},
#     'uim_config': {'aggfunc': 'sum',
#                    'weights': bm25_weight},
# }

In [6]:
preparer = DataProcessor(train, test, **mix_feat_params)
preparer.fit()

## User featuring

In [7]:
user_feat = pd.DataFrame(preparer.train_uim.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# решение "влоб" - из всех признаков сделать dummies
user_feat_final = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())

In [ ]:
# make pipeline

## Item featuring

In [9]:
item_feat = pd.DataFrame(preparer.train_uim.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

In [10]:
# решение "влоб" - из всех признаков сделать dummies
item_feat_final = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [10]:
# make pipeline

## LightFM baseline

In [ ]:
user_feat = pd.DataFrame(preparer.train_uim.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat_final = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())

In [ ]:
item_feat = pd.DataFrame(preparer.train_uim.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)
item_feat_final = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [11]:
model = LightFM(no_components=10,
                loss='warp', # 'bpr'
                learning_rate=0.05,
                item_alpha=0.1, # смещение по товару
                user_alpha=0.1,
                random_state=42)

model.fit((preparer.train_uim_sparse > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(preparer.train_uim),
          user_features=csr_matrix(user_feat_final.values).tocsr(),
          item_features=csr_matrix(item_feat_final.values).tocsr(),
          epochs=15)

In [76]:
item_index = np.arange(preparer.train_uim.columns.size)
predictions = model.predict(user_ids=6, item_ids=item_index,
                            user_features=csr_matrix(user_feat_final.values).tocsr(),
                            item_features=csr_matrix(item_feat_final.values).tocsr(),
                            num_threads=4)

In [77]:
#